<div style="text-align: right">
    <i>
        AMP 2019 (October 12) <br>
        Alëna Aksënova
    </i>
</div>

# _SigmaPie_ for subregular grammar induction

## Subregular languages in phonology

This toolkit is relevant for anyone who is working or going to work with subregualar grammars both from the perspectives of theoretical linguistics and formal language theory.

**Why theoretical linguistics should be interested in formal language theory?** <br>
_Formal language theory_ explains how potentially infinite stringsets, or _formal languages_,
can be generalized to grammars encoding the desired patterns and what properties those
grammars have. It also allows one to compare different grammars with respect to parameters such as expressivity.

**Chomsky hierarchy** aligns main classes of formal languages with respect to their expressive power.
  * **Regular** grammars are as powerful as finite state devices or regular expressions: they can "count" only until certain threshold (no $a^{n}b^{n}$ patterns);
  * **Context-free** grammars have access to potentially infinite _stack_ that allows them to reproduce patterns that involve center embedding;
  * **Mildly context-sensitive** grammars are powerful enough to handle some types of cross-serial dependencies such as copying;
  * **Context sensitive** grammars are restricted to a finitely long [memory tape](https://en.wikipedia.org/wiki/Punched_tape) encoding the pattern;
  * **Recursively enumerable** grammars are as powerful as any theoretically possible computer in this universe, they can use infinitely long memory tape.



<img src="images/chomhier.png" width="600">


Both phonology and morphology frequently display properties of regular languages.

**Phonology** does not require the power of center-embedding. For example, consider a harmony where the first vowel agrees with the last vowel, second vowel agrees with the pre-last vowel, etc.
    
    GOOD: "arugula", "tropicalization", "electrotelethermometer", etc.
    BAD:  any other word violating the rule.


While it is a theoretically possible pattern, harmonies of that types are unattested in natural languages.

**Morphology** avoids center-embedding as well. In [Aksënova et al. (2016)](https://www.aclweb.org/anthology/W16-2019) we show that it is possible to iterate prefixes with the meaning "after" in Russian. In Ilocano, where the same semantics is expressed via a circumfix, its iteration is prohibited.
    
    RUSSIAN: "zavtra" (tomorrow), "posle-zavtra" (the day after tomorrow), 
             "posle-posle-zavtra" (the day after the day after tomorrow), ...
    ILOCANO: "bigat" (morning), "ka-bigat-an" (the next morning),
             <*>"ka-ka-bigat-an-an" (the morning after the next one).


Moreover, typological review of patterns shows that phonology and morphology do not require the full power of regular languages. As an example of an unattested pattern, [Heinz (2011)](http://jeffreyheinz.net/papers/Heinz-2011-CPF.pdf) provides a language where a word must have an even number of vowels to be well-formed.


Regular languages can be sub-divided into another nested hierarchy of languages decreasing in their expressive power: **subregular hierarchy**.


<img src="images/subreg.png" width="250">


This tutorial and _SigmaPie_ toolkit currently contains functionality for the following classes:
  * strictly piecewise (SP);
  * strictly local (SL);
  * tier-based strictly local (TSL);
  * multiple tier-based strictly local (MTSL).

## Functionality of the toolkit

  * **Learners** extract grammars from stringsets.
  * **Scanners** evaluate strings with respect to a given grammar.
  * **Sample generators** generate stringsets for a given grammar.
  * **FSM constructors** translate subregular grammars to finite state machines.
  * **Polarity converters** switch negative grammars to positive, and vice versa.

In [2]:
%cd
%cd Desktop/SigmaPie/code/

from main import *

/home/alenaks
/home/alenaks/Desktop/SigmaPie/code


## Strictly piecewise languages

**Negative strictly $k$-piecewise (SP)** grammars prohibit occurrence of sequences of $k$ symbols at an arbitrary distance from each other. The value of $k$ defines the size of the window of the grammar, or the length of the longest sequence that the grammar can prohibit. Alternatively, if the grammar is positive, it lists subsequences that are allowed in well-formed words of the language.

    k = 2
    POLARITY: negative
    GRAMMAR:  ab, ba
    LANGUAGE: accaacc, cbccc, cccacaaaa, ...
              <*>accacba, <*>bcccacbb, <*>bccccccca, ...
              
              
In phonology, an example of an SP pattren is _tone plateauing_ considered in [Jardine (2015,](https://adamjardine.net/files/jardinecomptone-short.pdf) [2016)](https://adamjardine.net/files/jardine2016dissertation.pdf).
For example, in Luganda (Bantu) a low tone (L) cannot intervene in-between two high tones (H): L is changed to H in such configuration.
The prosodic domain cannot have more than one stretch of H tones.

**Luganda verb and noun combinations** (Hyman and Katamba (2010), cited by Jardine (2016))

  * /tw-áa-mú-láb-a, walúsimbi/ $\Rightarrow$ tw-áá-mu-lab-a, walúsimbi <br>
    ‘we saw him, Walusimbi’ <br>
    **HHLLL, LHLL**
    
  * /tw-áa-láb-w-a walúsimbi/ $\Rightarrow$ tw-áá-láb-wá wálúsimbi <br>
    ‘we were seen by Walusimbi’ <br>
    **HHHHHHLL**
    
  * /tw-áa-láb-a byaa=walúsimbi/ $\Rightarrow$ tw-áá-láb-á byáá-wálúsimbi <br>
    ‘we saw those of Walusimbi’ <br>
    **HHHHHHHHLL**
    
This pattern can be described using SP grammar $G_{SP_{neg}} = \{HLH\}$.

### Learning tone plateauing pattern

Let us say that `tone_plat` represents a "toy" example of tonal plateauing (TP) pattern.

In [3]:
tone_plat = ["LLLL", "HHLLL", "LHHHLL", "LLLLHHHH"]

Our goal will be to learn the generalization behind TP.

Negative and positive SP grammars are implemented in the package in the `SP()` class.

In [4]:
tp_pattern = SP()

### Attributes of SP grammars
  * `alphabet` (list) is the set of symbols that the grammar uses.
  * `grammar` (list of tuples) is the list of allowed or prohibited substructures of the language;
  * `k` (int) is the size of the locality window of the grammar, by default it is $2$;
  * `data` (list of string) is the learning sample;
  * `fsm` (FSM object) is the finite state device that corresponds to the grammar.
  
The initial step is to define the training sample and the alphabet.

In [5]:
tp_pattern.data = tone_plat
tp_pattern.alphabet = ["H", "L"]

By default, the locality window of the grammar is $2$.

In [6]:
print("Locality of the SP grammar:", tp_pattern.k)

Locality of the SP grammar: 2
Delimiters: ['>', '<']


All these attributes can be directly accessed. For example, let us change the locality of the window from $2$ to $3$:

In [7]:
tp_pattern.k = 3
print("Locality of the SP grammar:", tp_pattern.k)

Locality of the SP grammar: 3


### Methods for SP grammars
  * `check_polarity()` and `change_polarity()` display and switch the polarity of the grammar;
  * `learn()` extracts prohibited or allowed subsequences from the training sample;
  * `scan(string)` tells if a given string is well-formed with respect to a learned grammar;
  * `extract_alphabet()` collects alphabet based on the provided data;
  * `generate_sample(n, repeat)` generates $n$ strings based on the given grammar. By default, `repeat` is set to False, and repetitions of the generated strings are not allowed, but this parameter can be set to True.
  * `fsmize()` creates the corresponding FSM family by following the steps outlined in [Heinz&Rogers(2013)](https://www.aclweb.org/anthology/W13-3007).
  * `subsequences(string)` returns all $k$-piecewise subsequences of the given string;
  * `generate_all_ngrams()` generates all possible strings of the length $k$ based on the provided alphabet.

**Checking and changing polarity of the grammar**

By default, the grammars are positive. The polarity can be checked by running the `check_polarity` method:

In [8]:
print("Polarity of the grammar:", tp_pattern.check_polarity())

Polarity of the grammar: p


If the polarity needs to be changed, this can be done using the `change_polarity` method. It will automatically switch the grammar, if one is provided or already extracted, to the opposite one.

In [9]:
tp_pattern.change_polarity()
print("Polarity of the grammar:", tp_pattern.check_polarity())

Polarity of the grammar: n


**Learning the SP grammar**

Method `learn` extracts allowed or prohibited subsequences from the learning sample based on the polarity of the grammar and the locality window. Currently, $k=2$ and the grammar is negative.

In [10]:
tp_pattern.learn()
print("Extracted grammar:", tp_pattern.grammar)

Extracted grammar: [('H', 'L', 'H')]


Indeed, it learned the TP pattern!

$n$-grams are represented as tuples of strings, because in this case, elements of the alphabet are not restricted to characters, and it allows for other representations to be learned as well.

**Scanning strings and telling if they are part of the language**

Method `scan` takes  string as input and returns True or False depending on if the current string is contained in the language of the grammar:

In [11]:
tp = ["HHHLLL", "L", "HHL", "LLHLLL"]
no_tp = ["LLLLHLLLLH", "HLLLLLLH", "LLLHLLLHLLLHL"]

print("Tonal plateauing:")
for string in tp:
    print("String", string, "is in L(G):", tp_pattern.scan(string))
    
print("\nNo tonal plateauing:")
for string in no_tp:
    print("String", string, "is in L(G):", tp_pattern.scan(string))

Tonal plateauing:
String HHHLLL is in L(G): True
String L is in L(G): True
String HHL is in L(G): True
String LLHLLL is in L(G): True

No tonal plateauing:
String LLLLHLLLLH is in L(G): False
String HLLLLLLH is in L(G): False
String LLLHLLLHLLLHL is in L(G): False


**Generating a data sample**

Based on the learned grammar, a data sample of the desired size can be generated.

In [12]:
sample = tp_pattern.generate_sample(n = 10)
print("Sample:", sample)

Sample: ['', 'HLL', 'HL', 'HHH', 'LHHHHLLL', 'LLL', 'LHLL', 'H', 'LLHLL', 'L']


**Extracting subsequences**

Finally, this toolkit can be used also in order to extract subsequences from the input word by feeding it to the `subsequences` method.

In [25]:
tp_pattern.k = 3
print("k = 3:", tp_pattern.subsequences("regular"), "\n")
tp_pattern.k = 5
print("k = 5:", tp_pattern.subsequences("regular"))

k = 3: [('r', 'l', 'a'), ('u', 'a', 'r'), ('r', 'u', 'l'), ('r', 'e', 'g'), ('r', 'a', 'r'), ('g', 'a', 'r'), ('e', 'g', 'a'), ('e', 'g', 'r'), ('r', 'u', 'a'), ('e', 'g', 'l'), ('r', 'g', 'u'), ('g', 'u', 'r'), ('r', 'u', 'r'), ('u', 'l', 'a'), ('g', 'u', 'l'), ('r', 'g', 'l'), ('g', 'l', 'r'), ('e', 'a', 'r'), ('l', 'a', 'r'), ('r', 'e', 'l'), ('u', 'l', 'r'), ('r', 'g', 'r'), ('e', 'g', 'u'), ('e', 'l', 'a'), ('e', 'u', 'l'), ('r', 'l', 'r'), ('g', 'l', 'a'), ('e', 'u', 'r'), ('e', 'u', 'a'), ('r', 'g', 'a'), ('r', 'e', 'a'), ('r', 'e', 'u'), ('g', 'u', 'a'), ('e', 'l', 'r'), ('r', 'e', 'r')] 

k = 5: [('g', 'u', 'l', 'a', 'r'), ('r', 'e', 'g', 'u', 'l'), ('r', 'e', 'u', 'a', 'r'), ('r', 'g', 'u', 'l', 'a'), ('r', 'e', 'u', 'l', 'r'), ('r', 'g', 'l', 'a', 'r'), ('r', 'e', 'g', 'l', 'a'), ('r', 'e', 'l', 'a', 'r'), ('e', 'g', 'l', 'a', 'r'), ('r', 'e', 'g', 'u', 'a'), ('e', 'g', 'u', 'l', 'a'), ('r', 'g', 'u', 'l', 'r'), ('e', 'u', 'l', 'a', 'r'), ('r', 'e', 'u', 'l', 'a'), ('r', 'g'

While SP languages capture multiple long-distance processes such as tone plateauings or some harmonies, they are unable to capture local processes, or blocking effect.

## Strictly local languages

**Negative strictly $k$-local (SL)** grammars prohibit occurrence of consecutive substrings consisting of up to $k$ symbols. The value of $k$ in this case, defines the longest substring that cannot be present in a well-formed string of a language. Positive SL grammars defines substrings that can be present in the language.

Importantly, in order to define _first_ and _last_ elements, SL languages use delimiters (">" and "<") that indicate the beginning and the end of the string.

    k = 2
    POLARITY: positive
    GRAMMAR:  >a, ab, ba, b<
    LANGUAGE: ab, abab, abababab, ...
              <*>babab, <*>abaab, <*>bababba, ...
              
In phonology, very frequently changes involve adjacent segments, and the notion of _locality_ is therefore extremely important. The discussion of local processes in phonology can be found in [(Chandlee 2014)](http://dspace.udel.edu/bitstream/handle/19716/13374/2014_Chandlee_Jane_PhD.pdf).

  * `edges` (list of two characters) are the delimiters used by the grammar, the default value is ">" and "<";

**Acknowledgements** 
  * Thomas
  * Jeff
  * Ani

**Bibliography**

  * Make a reference to Chomsky (?) for the Chomsky hierarchy
  * Kaplan and Kay (1994)
  * Karttunen et al. (1992)
  * Shieber (1985)
  * Heinz (2011)
  * Aksenova et al (2016)
  * Jardine 2015 and 2016
  * Hyman and Katamba (2010)
  * Jeffrey Heinz and James Rogers. 2013. Learning subregular classes of languages with factored deterministic automata. In Proceedings of the 13th Meeting on the Mathematics of Language (MoL 13), pages 64–71, Sofia, Bulgaria. Association for Computational Linguistics.
  * Chandlee 2014